# Generate test data for package

## generate pgen files

In [ ]:
cd ~

In [ ]:
#R, 500 variants for each chromosome
for (i in 1:22){
  system(paste0("head -n 500 /home/simingz/causalTWAS/ukbiobank/ukb_chr", i, "_s40.22_snp.txt > test_chr", i, "_snp.txt"))

In [ ]:
# 2000 samples
head -n 2000 /home/simingz/causalTWAS/ukbiobank/ukbiobank_samples_s40.22.txt > test_samples.txt

In [ ]:
cmd = "cd /home/simingz/causalTWAS/ukbiobank/ukb_pgen_s40.22; plink2 --pfile ukb_chr{chrom}_s40.22_thinned --extract test_chr{chrom}_snp.txt --keep test_samples.txt --threads 5 --make-pgen --out test_chr{chrom}"
masterfile = "test_data-chr1to22.sh"
mf = open(masterfile, 'w')
for chrom in [str(i) for i in range(1,23)]:
  mf.write(cmd.format(chrom=chrom) + '\n')
mf.close()

In [ ]:
bash test_data-chr1to22.sh

In [1]:
#R, rename variant and sample
for (i in 1:22){
    df <- cbind(1:2000, 1:2000, sample(1:2, 2000, replace =T))
    colnames(df) <- c("#FID", "IID", "SEX")
    write.table(df , file= paste0("example_chr", i, ".psam") , row.names=F, col.names=T, sep="\t", quote = F)
}
for (i in 1:22){
    df <- data.table::fread(paste0("example_chr", i, ".pvar"), header =T)
    df$POS <- 1:nrow(df)
    df$ID <- paste0("rs", 1:nrow(df) + i * 1000)
    df$REF <- sapply(1:nrow(df), function(x) c("A", "T")[sample(1:2,1)])
    df$ALT <- sapply(1:nrow(df), function(x) c("C", "G")[sample(1:2,1)])
    write.table( df , file= paste0("example_chr", i, ".pvar") , row.names=F, col.names=T, sep="\t", quote = F)
}

## generate fusion weight files

In [ ]:
# R, wgt.RDat file and .pos file
# 20 genes, 1 gene in each of first 20 chromosomes
# first a few snps are eqtls, selected to have nonzero lasso weights.
wgtdir <- "/project2/mstephens/causalTWAS/fusion_weights/Adipose_Subcutaneous"
posfile <- paste0(wgtdir, ".pos")
posdf <- data.table::fread(posfile)[1:20]

for (i in 1:nrow(posdf)){
    load(file.path(dirname(wgtdir), posdf$WGT[i]))
    wgt.matrix <- wgt.matrix[abs(wgt.matrix[, "lasso"]) > 0, ]
    snps <- data.table::fread(paste0("/home/simingz/ctwas/inst/extdata/example_genotype_files/example_chr", i, ".pvar"))[1:nrow(wgt.matrix),]
    rownames(wgt.matrix) <- snps$ID
    snps$CM <- 0
    snps <- snps[, c("#CHROM", "ID", "CM", "POS","ALT" ,"REF")]
    colnames(snps) <- paste0("V", 1:6)
    snps <- data.frame(snps)
    N.tot <- 100
    save(wgt.matrix, snps, cv.performance, hsq, hsq.pv, N.tot, file = paste0("/home/simingz/ctwas/inst/extdata/example_fusion_weights/Tissue/Tissue.gene", i, ".wgt.RDat"))
}

posdf$PANEL <- "Tissue"
posdf$WGT <- paste0("Tissue/Tissue.gene", 1:20, ".wgt.RDat")
posdf$ID <- paste0("gene", 1:20)
posdf$CHR <- 1:20
posdf$P0 <- 1
posdf$P1 <- 30
posdf$N <- 100
write.table(posdf , file= paste0("/home/simingz/ctwas/inst/extdata/example_fusion_weights/Tissue.pos"), row.names=F, col.names=T, sep="\t", quote = F)

## generate region files

In [ ]:
# 1 regions for each chr
regionlist <- list()
for (i in 1:22){
    regionlist[[i]] <- rbind(data.frame("chr" = i, start = 0, stop = 5),
                             data.frame("chr" = i, start = seq(5, 455, 30), stop = seq(35,505,30)))
                            
}
regions <- do.call(rbind, regionlist)
write.table(regions , file= "~/ctwas/inst/extdata/example_regions.bed"  , row.names=F, col.names=T, sep="\t", quote = F)

## generate LD R matrices

In [ ]:
setwd("/home/simingz/ctwas/inst/extdata/example_ld_R/")

# specify LD reference
ldref_files <- system.file("extdata/example_genotype_files", paste0("example_chr", 1:22, ".pgen"),package = "ctwas")

# specify LD regions
ld_regions_custom = system.file("extdata", "example_regions.bed", package = "ctwas")

# specify output
file_stem <- "example_chr"

####################

if (is.null(ld_regions_custom)) {
  #ld_regions <- match.arg(ld_regions)
  regionfile <- system.file("extdata", "ldetect", paste0(ld_regions, ".bed"), package = "ctwas")
} else {
  regionfile <- ld_regions_custom
}
reg <- read.table(regionfile, header = T, stringsAsFactors = F)
if (is.character(reg$chr)) {
  reg$chr <- readr::parse_number(reg$chr)
}
reg <- reg[order(reg$chr, reg$start), ]

ld_pvarfs <- sapply(ldref_files, ctwas:::prep_pvar)

for (b in 1:length(ld_pvarfs)) {
  print(paste0("chromosome ", b))
  ld_pvarf <- ld_pvarfs[b]
  snpinfo <- ctwas:::read_pvar(ld_pvarf)
  if (unique(snpinfo$chrom) != b) {
    stop("Input genotype file not split by chromosome or not in correct order")
  }
  regions <- reg[reg$chr == b, ]
  ld_pgen <- ctwas:::prep_pgen(pgenf = ldref_files[b], ld_pvarfs[b])
  Rinfo <- NULL
  for (rn in 1:nrow(regions)) {
    print(rn)
    p0 <- regions[rn, "start"]
    p1 <- regions[rn, "stop"]
    sidx <- which(snpinfo$chrom == b & snpinfo$pos >= p0 & snpinfo$pos < p1)
    sid <- snpinfo[sidx, "id"]
    X.g <- ctwas:::read_pgen(ld_pgen, variantidx = sidx)
    R_snp <- Rfast::cora(X.g)
    R_snp_anno <- snpinfo[sidx, ]
    saveRDS(R_snp, file=paste0(file_stem, b, ".R_snp.", p0, "_", p1, ".RDS"), version = 2)
    write.table(R_snp_anno, file= paste0(file_stem, b, ".R_snp.", p0, "_", p1, ".Rvar"),
                row.names=F, col.names=T, sep="\t", quote = F)
    Rinfo <- rbind(Rinfo,  c(b, rn,  p0, p1, system.file("extdata/example_ld_R", paste0(file_stem, b, ".R_snp.", p0, "_", p1, ".RDS"), package ="ctwas")))
  }
  colnames(Rinfo) <- c("chrom", "region_name", "start", "stop", "RDS_file")
  write.table(Rinfo, file= paste0(file_stem, b, "_ld_R_info.txt"), row.names=F, col.names=T, sep="\t", quote = F)
}

# example predict.db

In [ ]:
setwd("/home/simingz/ctwas/inst/extdata/example_fusion_weights/Tissue")

setwd("/home/simingz/causalTWAS/fusion_weights/Adipose_Subcutaneous")
suppressPackageStartupMessages(library(RSQLite))
suppressPackageStartupMessages(library(tidyverse))
suppressPackageStartupMessages(library(data.table))

make_df <- function(file) {
  load(file)  
   weights <- data.frame(wgt.matrix) 
  snps <- data.frame(snps) 
  weights$rsid <- rownames(weights)
  weights$gene <- strsplit(file, split = "\\." )[[1]][2]
  weights$chromosome <- snps[match(weights$rsid, snps$V2),]$V1 
  weights$position <- snps[match(weights$rsid, snps$V2),]$V4
  weights$ref_allele <- snps[match(weights$rsid, snps$V2),]$V6
  weights$eff_allele <- snps[match(weights$rsid, snps$V2),]$V5
  weights %>% filter(lasso != 0) %>% select(gene, rsid, chromosome, position, ref_allele, eff_allele, lasso)
  
}

files <- list.files(pattern = "\\.RDat")

pre_weights = glue::glue("pre_weights.txt")
write.table(make_df(files[1]), pre_weights, sep = "\t", quote = FALSE, row.names = FALSE)
for(i in 2:length(files)) {
  write.table(make_df(files[i]), pre_weights, append = TRUE, sep = "\t", quote = FALSE, col.names = FALSE, row.names = FALSE)
}

# Add varIDs

weights <- fread("pre_weights.txt")
weights$varID <- paste(paste("chr", weights$chromosome, sep = ""), weights$position, weights$ref_allele, weights$eff_allele, "b37", sep = "_")
weights <- weights %>% select(gene, rsid, varID, ref_allele, eff_allele, lasso) %>% rename(weight = lasso)


# Make Extra Table

extra <- weights %>% group_by(gene) %>% summarise(n.snps.in.model = n())
extra$genename <- NA
extra$pred.perf.R2 <- NA
extra$pred.perf.pval <- NA
extra$pred.perf.qval <- NA
extra <- extra[c(1, 3, 2, 4, 5, 6)]

# Write to SQLite Database

# model_db = "/home/simingz/ctwas/inst/extdata/example_tissue.db"
model_db = "/home/simingz/causalTWAS/fusion_weights/Adipose_Subcutaneous.db"
conn <- dbConnect(RSQLite::SQLite(), model_db)
dbWriteTable(conn, "weights", weights)
dbWriteTable(conn, "extra", extra)

dbListTables(conn)
dbGetQuery(conn, 'SELECT * FROM weights') %>% head
dbGetQuery(conn, 'SELECT * FROM extra') %>% head

dbDisconnect(conn)

# need to remove pre-weights.txt